In [1]:
!pip install psutil

In [28]:
!pip install tokenizers

  Obtaining dependency information for tokenizers from https://files.pythonhosted.org/packages/44/69/d21eb253fa91622da25585d362a874fa4710be600f0ea9446d8d0217cec1/tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)


In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, TrainingArguments, Trainer
from tokenizers import trainers
from datasets import load_dataset
from emoji import demojize
from langdetect import detect
import tempfile
import torch
import os
import re
import spacy
import logging

In [20]:
os.chdir(r"C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support")
print(os.getcwd())

C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support


In [35]:
# Загружаем оригинальный токенизатор
# Загружаем оригинальный токенизатор
original_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

# Загружаем токенизатор в виде объекта tokenizers
tokenizer = original_tokenizer.backend_tokenizer

# Создаём новый тренер для дообучения
special_tokens = [original_tokenizer.unk_token, original_tokenizer.pad_token, original_tokenizer.bos_token, original_tokenizer.eos_token]

trainer = trainers.BpeTrainer(show_progress=True, vocab_size=original_tokenizer.vocab_size*2, special_tokens=special_tokens)
file_paths = ["./data/ru_taiga-ud-train.txt", "./data/lt_full_corpus.txt", "./data/en_eslspok-ud-train.txt"]


# try:
    # Обучаем токенизатор
tokenizer.train(file_paths, trainer)
# finally:
#     print(f"tmp {tmp_path} file deleting")
#     os.remove(tmp_path)
# Сохраняем обновленный токенизатор
tokenizer.save("gemma2_russian_tokenizer.json")

In [ ]:
# with tempfile.NamedTemporaryFile(mode="w+", delete=False, encoding="utf-8") as temp_file:
#     # Читаем данные из всех файлов и записываем их во временный файл
#     for file_path in file_paths:
#         try:
#             with open(file_path, "r", encoding="utf-8") as f:
#                 temp_file.write(f.read() + "\n")  # Добавляем перенос строки между файлами
#         except UnicodeDecodeError as e:
#             print(file_path, "is UnicodeDecodeError")
#             raise e
#     # Перемещаем указатель в начало временного файла, чтобы прочитать его содержимое
#     temp_file.flush()
#     tmp_path = temp_file.name
    # Обучаем модель на корпусе

In [37]:
model_name = "google/gemma-2-2b"

In [38]:
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
# Вывод параметров конфигурации


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [39]:

# sp = spm.SentencePieceProcessor()
text = "Привет, как твои дела? Labas, kaip sekasi?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

TypeError: 'tokenizers.Tokenizer' object is not callable

In [14]:
print(config)

Gemma2Config {
  "_name_or_path": "google/gemma-2-2b",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 8,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "query_pre_attn_scalar": 256,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "torch_dtype": "float32",
  "transformers_version": "4.48.1",
  "use_cache": true,
  "vocab_size": 256000
}



In [5]:
# # Кастомная конфигурация
# 
# class Gemma2NormalizedConfigOnnx:
#     def __init__(self, config):
#         self.hidden_size = config.hidden_size
#         self.num_attention_heads = config.num_attention_heads
#         self.num_hidden_layers = config.num_hidden_layers
#         self.vocab_size = config.vocab_size
#         # self.max_position_embeddings = config.max_position_embeddings
#         
#         
# class Gemma2OnnxConfig(OnnxConfig):
#     NORMALIZED_CONFIG_CLASS = Gemma2NormalizedConfigOnnx
#     @property
#     def inputs(self) -> Mapping[str, Mapping[int, str]]:
#         return {"input_ids": {0: "batch_size", 1: "sequence_length"}}
# 
#     @property
#     def outputs(self) -> Mapping[str, Mapping[int, str]]:
#         return {"logits": {0: "batch_size", 1: "sequence_length"}}
# 
#     @property
#     def default_onnx_opset(self) -> int:
#         return 14


In [6]:
# # Экспорт модели с использованием кастомной конфигурации
# from pathlib import Path
# onnx_model_path =  Path(r"C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support\models\gemma2")
# config = Gemma2OnnxConfig(config)
# export(
#     model=model_,
#     config=config,
#     output=onnx_model_path,  # Путь для сохранения ONNX модели
#     device="cpu"  # Поскольку у вас нет GPU
# )

IndexError: tuple index out of range

In [7]:
# custom_onnx_configs={
#     "gemma2": config,
# }
# 
# main_export(
#     model_name,
#     custom_onnx_configs=custom_onnx_configs,
#     output=onnx_model_path,  # Путь для сохранения ONNX модели
#     device="cpu",
#     trust_remote_code=True,
#     no_post_process=True,# Поскольку у вас нет GPU
# )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ONNX custom configs for: gemma2
Submodels to export: model


ValueError: Trying to export a custom model, but could not find as many custom ONNX configs as the number of submodels to export. Please specifiy the fn_get_submodels argument, that should return a dictionary of submodules with as many items as the provided custom_export_configs dictionary.

In [ ]:
# # tokenizer = AutoTokenizer.from_pretrained(model_name)
# inputs = tokenizer("Пример текста", return_tensors="pt")
# 
# # Прогон данных через модель
# outputs = model_(**inputs)
# 
# # Вывод доступных выходов
# print(outputs)

In [9]:
# # Токенизация данных
# def tokenize_function(examples):
#     return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256, return_tensors="pt")
# # 
# model = ORTModelForCausalLM.from_pretrained(model_name,  export=True)

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [15]:
text = "Hi. How are you?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(inputs.input_ids, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

TypeError: 'bool' object is not callable

In [8]:
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
    ],
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log'),
    record_shapes=True,
    with_stack=True
) as prof:
    text = "Hi. 2+2=?"
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=50)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=50))

Hi. 2+2=?

[User 0001]

Hi. I'm new to this forum. I'm a 2+2 player. I'm 21 years old. I'm
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                         aten::linear         0.06%     190.791ms        98.86%      331.798s      43.169ms          7686  
                                         aten::matmul         0.13%     438.298ms        98.74%      331.405s      42.884ms          7728  
                                             aten::mm        98.54%      330.734s        98.55%      330.742s      43.032ms          7686  
                                             aten::to         0.05%  

In [6]:
text = "Hi. 2+2=?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(inputs.input_ids, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hi. 2+2=?

[User 0001]

Hi. I'm new to this forum. I'm a 2+2 player. I'm 21 years old. I'm


In [7]:
text = "What is 2*2^5+2^8/7?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(inputs.input_ids, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What is 2*2^5+2^8/7?

[User 0001]

I'm not sure how to do this problem. I know that 2^5=32 and 2


In [ ]:
ds = load_dataset("devngho/culturax-mini-nonshuffled", 'ru')
output_dir = "models/gemma2"
os.makedirs(output_dir, exist_ok=True)

In [6]:
def initialize_spacy_models():
    """
    Инициализирует SpaCy модели для каждого языка.
    """
    return {
        "en": spacy.load("en_core_web_sm"),
        "ru": spacy.load("ru_core_news_sm"),
        "lt": spacy.load("lt_core_news_sm")
    }

def detect_language(text):
    """
    Определяет язык текста.
    :param text: Входной текст
    :return: Код языка (ru, en, lt)
    """
    return detect(text)


In [9]:
spacy_models = initialize_spacy_models()


def preprocess_text(text):
    """
    Обработка текста: приведение к нижнему регистру, очистка, токенизация, лемматизация, удаление стоп-слов.
    :param text: входной текст
    :param lang: язык текста (ru, en, lt)
    :return: список лемматизированных токенов
    """
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление HTML-тегов
    text = re.sub(r"<.*?>", "", text)

    # Удаление эмодзи
    text = demojize(text)

    # Удаление лишних символов
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    
    lang = detect_language(text)
    # Токенизация и лемматизация
    tokens = []
    if lang in ["ru", "en", "lt"]:
        doc = spacy_models[lang](text)
        tokens = [
            token.lemma_ 
            for token in doc 
            if token.text.lower() not in spacy_models[lang].Defaults.stop_words and not token.is_punct
        ]
        logging.info(f"Лемматизация для языка {lang}: {tokens}")

    return tokens

def tokenize_function(examples):
        return tokenizer(examples["lt_text"], padding="max_length", truncation=True, max_length=128)

In [10]:
def tokenize_text(dataset, column_names=['text']):
    for column_name in column_names:
        for text in dataset[column_name]:
             print(f"Обработанный текст: {preprocess_text(text)}")
        print("-")
            

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe):
        """
        Инициализация кастомного датасета
        :param dataframe: Pandas DataFrame с путями к изображениям и метками
        :param transform: Преобразования для изображений (например, аугментации)
        """
        self.dataframe = dataframe

    def __len__(self):
        # Возвращаем количество элементов в датасете
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Получаем путь к изображению и метку по индексу
        image = Image.open(io.BytesIO(self.dataframe.iloc[idx, 0]['bytes'])).convert("RGB")   # столбец с путём к изображению
        label = self.dataframe.iloc[idx, 9]
        mask = Image.open(self.dataframe.iloc[idx, 8])

        # Применяем преобразования, если они указаны
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask, label

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,          # Папка для сохранения модели
    evaluation_strategy="epoch",  # Оценка каждые эпохи
    learning_rate=2e-5,            # Скорость обучения
    per_device_train_batch_size=8, # Размер батча
    num_train_epochs=3,            # Количество эпох
    save_steps=10_000,             # Шаги сохранения
    save_total_limit=2,            # Максимум сохранений
    logging_dir="logs",           # Логи
    logging_steps=500,             # Шаг логирования
    push_to_hub=False              # Отключение публикации в Hugging Face
)

trainer = Trainer(
    model=model,                   # Модель
    args=training_args,            # Параметры обучения
    train_dataset=tokenized_dataset, # Тренировочные данные
    tokenizer=tokenizer            # Токенизатор
)